<a href="https://colab.research.google.com/github/alexandergribenchenko/Data_Science_Finanzas/blob/main/PY_Creditos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01. Librerías

In [2]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 43.2 MB/s eta 0:00:00


In [75]:
import fitz  # PyMuPDF
import pandas as pd
import re

# 02. PDF to Text

In [82]:
# Ruta del archivo PDF
pdf_path = "BBVA_02_Hipotecario_Vivienda_100M_05A_20020321.pdf"
# pdf_path = "BBVA_02_Hipotecario_Vivienda_100M_10A_20020321.pdf"
# pdf_path = "BBVA_02_Hipotecario_Remodelacion_100M_05A_20020321.pdf"
# pdf_path = "BBVA_02_Hipotecario_Remodelacion_100M_10A_20020321.pdf"
# pdf_path = "BBVA_01_Libre_Inversion_05M_01A_20020317.pdf"


# Leer el PDF y extraer texto
doc = fitz.open(pdf_path)
texto = []
for pagina in doc:
    texto.append(pagina.get_text("text"))

texto_concatenado = "\n".join(texto)
texto_concatenado

'Simulador credito de vivienda\nCrédito hipotecario\nCuota mensual con seguros desde:\n$2.309.309\nValor de la vivienda\n$200.000.000\nCouta inicial necesaria\n$100.000.000\nPlazo del crédito\n5 años\nIncluir seguros\nSi\nValor que vas a solicitar\n$ 100.000.000\nValor total intereses\n$ 33.949.303\nValor total seguro de vida\n$ 587.971\nValor total seguro incendio y terremoto\n$ 3.570.043\nValor total a pagar + seguros\n$ 133.949.303\nTasa de interés efectiva anual\n12.86%\nValor total unificado (VTU%)\n14.41%\nEste cálculo es una simulación, es aproximado e informativo y puede variar.\nRecuerda que la UVR varía diariamente según el IPC y la tasa será la vigente al día\ndel desembolso. Este resultado no compromete al Banco, ni constituye una\nasesoría.\n\nPlan de pagos\nEste plan de pagos incluye un beneficio con condiciones\nAprovecha esta tasa preferencial mientras mantengas y recibas tu salario o pensión en tu cuenta de \nnómina o pensional en BBVA.\nMes\nCapital\nIntereses\nCuota 

# 03. Validador regex

In [83]:
import re

def extraer_valores(texto):
    """Extrae valores clave de un texto relacionado con crédito de consumo."""

    patrones = {
        "Plazo (años)": r"Plazo del crédito\s*(\d+)\s*años",
        "Tasa de interés (%)": r"Tasa de interés efectiva anual\s*([\d,.]+)%",
        "VTU (%)": r"Valor total unificado \(VTU%\)\s*([\d,.]+)%",
        "Valor solicitado": r"Valor que vas a solicitar\s*\$ ([\d,.]+)",
        "Total intereses": r"Valor total intereses\s*\$ ([\d,.]+)",
        "Total seguro de vida": r"Valor total seguro de vida\s*\$ ([\d,.]+)",
        "Total seguro incendio/terremoto": r"Valor total seguro incendio y terremoto\s*\$ ([\d,.]+)",
        "Total a pagar + seguros": r"Valor total a pagar \+ seguros\s*\$ ([\d,.]+)"
    }

    valores_extraidos = {}
    for key, patron in patrones.items():
        match = re.search(patron, texto, re.IGNORECASE)
        valores_extraidos[key] = match.group(1) if match else "No encontrado"

    return valores_extraidos

valores = extraer_valores(texto_concatenado)
valores

{'Plazo (años)': '5',
 'Tasa de interés (%)': '12.86',
 'VTU (%)': '14.41',
 'Valor solicitado': '100.000.000',
 'Total intereses': '33.949.303',
 'Total seguro de vida': '587.971',
 'Total seguro incendio/terremoto': '3.570.043',
 'Total a pagar + seguros': '133.949.303'}

# 04. Raw dataframes créditos

## 04.01. Complete code: Credito hipotecario

In [86]:
def extraer_datos_pdf(pdf_path):
    """Extrae información financiera y tabla de pagos de un archivo PDF."""
    doc = fitz.open(pdf_path)
    texto = "\n".join([pagina.get_text("text") for pagina in doc])

    # Detectar dinámicamente el tipo de crédito
    patron_tipo_credito = re.search(r"(Crédito\s+[a-zA-Záéíóúñ]+)", texto, re.IGNORECASE)
    tipo_credito = patron_tipo_credito.group(0) if patron_tipo_credito else "Desconocido"

    # Expresiones regulares para valores clave
    patrones = {
        "Plazo (años)": r"Plazo del crédito\s*(\d+)\s*años",
        "Tasa de interés (%)": r"Tasa de interés efectiva anual\s*([\d,.]+)%",
        "VTU (%)": r"Valor total unificado \(VTU%\)\s*([\d,.]+)%",
        "Valor solicitado": r"Valor que vas a solicitar\s*\$ ([\d,.]+)",
        "Total intereses": r"Valor total intereses\s*\$ ([\d,.]+)",
        "Total seguro de vida": r"Valor total seguro de vida\s*\$ ([\d,.]+)",
        "Total seguro incendio/terremoto": r"Valor total seguro incendio y terremoto\s*\$ ([\d,.]+)",
        "Total a pagar + seguros": r"Valor total a pagar \+ seguros\s*\$ ([\d,.]+)"
    }

    valores_extraidos = {}
    for key, patron in patrones.items():
        match = re.search(patron, texto, re.IGNORECASE)
        valores_extraidos[key] = float(match.group(1).replace('.', '').replace(',', '.')) if match else 0.0

    # Expresión regular para extraer la tabla de pagos
    patron_fila = re.compile(r"(\d+)\s+\$ ([\d,.]+)\s+\$ ([\d,.]+)\s+\$ ([\d,.]+)\s+\$ ([\d,.]+)\s+\$ ([\d,.]+)\s+\$ ([\d,.]+)\s+\$ ([\d,.]+)")

    data = []
    for match in patron_fila.findall(texto):
        data.append([
            pdf_path,  # Nombre del archivo PDF
            tipo_credito,  # Tipo de crédito extraído
            valores_extraidos["Plazo (años)"],
            valores_extraidos["Tasa de interés (%)"],
            valores_extraidos["VTU (%)"],
            valores_extraidos["Valor solicitado"],
            valores_extraidos["Total intereses"],
            valores_extraidos["Total seguro de vida"],
            valores_extraidos["Total seguro incendio/terremoto"],
            valores_extraidos["Total a pagar + seguros"],
            int(match[0]),  # Mes
            float(match[1].replace('.', '').replace(',', '.')),  # Capital
            float(match[2].replace('.', '').replace(',', '.')),  # Intereses
            float(match[3].replace('.', '').replace(',', '.')),  # Cuota sin seguros
            float(match[4].replace('.', '').replace(',', '.')),  # Seguro de vida
            float(match[5].replace('.', '').replace(',', '.')),  # Seguro incendio/terremoto
            float(match[6].replace('.', '').replace(',', '.')),  # Cuota con seguros
            float(match[7].replace('.', '').replace(',', '.'))   # Saldo
        ])

    # Columnas del DataFrame
    columnas = ["Archivo", "Tipo de crédito", "Plazo (años)", "Tasa de interés (%)", "VTU (%)",
                "Valor solicitado", "Total intereses", "Total seguro de vida",
                "Total seguro incendio/terremoto", "Total a pagar + seguros",
                "Mes", "Capital", "Intereses", "Cuota sin seguros",
                "Seguro de vida", "Seguro incendio/terremoto", "Cuota con seguros", "Saldo"]

    return pd.DataFrame(data, columns=columnas)

def procesar_pdfs(lista_archivos):
    """Procesa una lista de archivos PDF y devuelve un único DataFrame concatenado."""
    dataframes = [extraer_datos_pdf(pdf) for pdf in lista_archivos]
    return pd.concat(dataframes, ignore_index=True)

# EJEMPLO DE USO
lista_pdfs = [
    "BBVA_02_Hipotecario_Vivienda_100M_05A_20020321.pdf",
    "BBVA_02_Hipotecario_Vivienda_100M_10A_20020321.pdf",
    "BBVA_02_Hipotecario_Remodelacion_100M_05A_20020321.pdf",
    "BBVA_02_Hipotecario_Remodelacion_100M_10A_20020321.pdf"
]

df_credito_hipotecario = procesar_pdfs(lista_pdfs)
df_credito_hipotecario


,Archivo,Tipo de crédito,Plazo (años),Tasa de interés (%),VTU (%),Valor solicitado,Total intereses,Total seguro de vida,Total seguro incendio/terremoto,Total a pagar + seguros,Mes,Capital,Intereses,Cuota sin seguros,Seguro de vida,Seguro incendio/terremoto,Cuota con seguros,Saldo
0,BBVA_02_Hipotecario_Vivienda_100M_05A_20020321...,Crédito hipotecario,5.0,1286.0,1441.0,100000000.0,33949303.0,587971.0,3570043.0,133949303.0,1,1219235.0,1013248.0,2232483.0,17325.0,59501.0,2309309.0,98780766.0
1,BBVA_02_Hipotecario_Vivienda_100M_05A_20020321...,Crédito hipotecario,5.0,1286.0,1441.0,100000000.0,33949303.0,587971.0,3570043.0,133949303.0,2,1231588.0,1000894.0,2232483.0,17114.0,59501.0,2309097.0,97549177.0
2,BBVA_02_Hipotecario_Vivienda_100M_05A_20020321...,Crédito hipotecario,5.0,1286.0,1441.0,100000000.0,33949303.0,587971.0,3570043.0,133949303.0,3,1244067.0,988415.0,2232483.0,16900.0,59501.0,2308884.0,96305110.0
3,BBVA_02_Hipotecario_Vivienda_100M_05A_20020321...,Crédito hipotecario,5.0,1286.0,1441.0,100000000.0,33949303.0,587971.0,3570043.0,133949303.0,4,1256673.0,975810.0,2232483.0,16685.0,59501.0,2308668.0,95048437.0
4,BBVA_02_Hipotecario_Vivienda_100M_05A_20020321...,Crédito hipotecario,5.0,1286.0,1441.0,100000000.0,33949303.0,587971.0,3570043.0,133949303.0,5,1269406.0,963077.0,2232483.0,16467.0,59501.0,2308451.0,93779031.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,BBVA_02_Hipotecario_Remodelacion_100M_10A_2002...,Crédito de,10.0,1317.0,1455.0,100000000.0,75204440.0,1303108.0,7140085.0,175204440.0,116,1386656.0,73370.0,1460027.0,1374.0,59501.0,1520902.0,5693082.0
356,BBVA_02_Hipotecario_Remodelacion_100M_10A_2002...,Crédito de,10.0,1317.0,1455.0,100000000.0,75204440.0,1303108.0,7140085.0,175204440.0,117,1401027.0,59000.0,1460027.0,1105.0,59501.0,1520632.0,4292056.0
357,BBVA_02_Hipotecario_Remodelacion_100M_10A_2002...,Crédito de,10.0,1317.0,1455.0,100000000.0,75204440.0,1303108.0,7140085.0,175204440.0,118,1415546.0,44480.0,1460027.0,833.0,59501.0,1520360.0,2876509.0
358,BBVA_02_Hipotecario_Remodelacion_100M_10A_2002...,Crédito de,10.0,1317.0,1455.0,100000000.0,75204440.0,1303108.0,7140085.0,175204440.0,119,1430216.0,29810.0,1460027.0,558.0,59501.0,1520086.0,1446293.0


## 04.02.  Complete code: Credito libre inversión

In [85]:
import fitz  # PyMuPDF
import pandas as pd
import re

def extraer_datos_pdf_libre_inversion(pdf_path):
    """Extrae información financiera y tabla de pagos de un archivo PDF de crédito de libre inversión."""
    doc = fitz.open(pdf_path)
    texto = "\n".join([pagina.get_text("text") for pagina in doc])

    # Detectar tipo de crédito
    patron_tipo_credito = re.search(r"(Libre Inversión)", texto, re.IGNORECASE)
    tipo_credito = patron_tipo_credito.group(0) if patron_tipo_credito else "Desconocido"

    # Expresiones regulares para valores clave con los nombres exactos
    patrones = {
        "Valor del crédito": r"Valor del crédito\s*\n+Plazo del crédito\s*\n+\$?\s*([\d.,]+)",
        "Plazo del crédito": r"Plazo del crédito\s*\n+\$?\s*[\d.,]+\s*\n+(\d+)\s*meses",
        "Tasa Efectiva Anual (E.A) desde": r"Tasa Efectiva Anual \(E\.A\) desde\s*\n(?:.*\n)*?([\d,.]+)%",
        "VTU Porcentual": r"VTU Porcentual\s*\n(?:Tasa Mensual Vencida.*?\n)?\s*[\d,.]+%\s*\n([\d,.]+)%",
        "Tasa Mensual Vencida (M.V) desde":r"VTU Porcentual\s*\nTasa Mensual Vencida \(M\.V\) desde\s*\n[\d,.%]+\s*\n[\d,.%]+\s*\n([\d,.%]+)%",
        "Valor por intereses": r"Valor por intereses\s*\n(?:.*\n)*?\$\s*([\d,.]+)",
        "Valor por seguro de vida": r"Valor por seguro de vida\s*\n\$\s*[\d,.]+\s*\n\$\s*([\d,.]+)",
        "Total": r"Total\s*\n+\$?\s*([\d.,]+)"
    }

    valores_extraidos = {}
    for key, patron in patrones.items():
        match = re.search(patron, texto, re.IGNORECASE)
        valores_extraidos[key] = float(match.group(1).replace('.', '').replace(',', '.')) if match else 0.0

    # Expresión regular para extraer la tabla de pagos
    patron_fila = re.compile(r"(\d+)\s+\$ ([\d,.]+)\s+\$ ([\d,.]+)\s+\$ ([\d,.]+)\s+\$ ([\d,.]+)\s+\$ ([\d,.]+)")

    data = []
    for match in patron_fila.findall(texto):
        data.append([
            pdf_path,  # Nombre del archivo PDF
            tipo_credito,  # Tipo de crédito extraído
            valores_extraidos["Valor del crédito"],
            valores_extraidos["Plazo del crédito"],
            valores_extraidos["Tasa Efectiva Anual (E.A) desde"],
            valores_extraidos["VTU Porcentual"],
            valores_extraidos["Tasa Mensual Vencida (M.V) desde"],
            valores_extraidos["Valor por intereses"],
            valores_extraidos["Valor por seguro de vida"],
            valores_extraidos["Total"],
            int(match[0]),  # Mes
            float(match[1].replace('.', '').replace(',', '.')),  # Cuota Mensual
            float(match[2].replace('.', '').replace(',', '.')),  # Intereses mes vencido
            float(match[3].replace('.', '').replace(',', '.')),  # Seguro de vida
            float(match[4].replace('.', '').replace(',', '.')),  # Abono a capital
            float(match[5].replace('.', '').replace(',', '.'))   # Saldo
        ])

    # Columnas del DataFrame
    columnas = ["Archivo", "Tipo de crédito", "Valor del crédito", "Plazo del crédito",
                "Tasa Efectiva Anual (E.A) desde", "VTU Porcentual", "Tasa Mensual Vencida (M.V) desde",
                "Valor por intereses", "Valor por seguro de vida", "Total",
                "Mes", "Cuota Mensual", "Intereses mes vencido", "Seguro de vida", "Abono a capital", "Saldo"]

    return pd.DataFrame(data, columns=columnas)

def procesar_pdfs_libre_inversion(lista_archivos):
    """Procesa una lista de archivos PDF de crédito de libre inversión y devuelve un único DataFrame concatenado."""
    dataframes = [extraer_datos_pdf_libre_inversion(pdf) for pdf in lista_archivos]
    return pd.concat(dataframes, ignore_index=True)

# 🔥 EJEMPLO DE USO
lista_pdfs_libre = [
    "BBVA_01_Libre_Inversion_05M_01A_20020317.pdf",
    "BBVA_01_Libre_Inversion_05M_04A_20020317.pdf"
]

df_credito_libre_inversion = procesar_pdfs_libre_inversion(lista_pdfs_libre)
df_credito_libre_inversion


,Archivo,Tipo de crédito,Valor del crédito,Plazo del crédito,Tasa Efectiva Anual (E.A) desde,VTU Porcentual,Tasa Mensual Vencida (M.V) desde,Valor por intereses,Valor por seguro de vida,Total,Mes,Cuota Mensual,Intereses mes vencido,Seguro de vida,Abono a capital,Saldo
0,BBVA_01_Libre_Inversion_05M_01A_20020317.pdf,Libre Inversión,5000000.0,12.0,23.4,24.79,1.77,592914.0,31866.0,5624780.0,1,470825.0,88380.0,4750.0,377695.0,4622305.0
1,BBVA_01_Libre_Inversion_05M_01A_20020317.pdf,Libre Inversión,5000000.0,12.0,23.4,24.79,1.77,592914.0,31866.0,5624780.0,2,470467.0,81704.0,4391.0,384371.0,4237934.0
2,BBVA_01_Libre_Inversion_05M_01A_20020317.pdf,Libre Inversión,5000000.0,12.0,23.4,24.79,1.77,592914.0,31866.0,5624780.0,3,470101.0,74910.0,4026.0,391165.0,3846769.0
3,BBVA_01_Libre_Inversion_05M_01A_20020317.pdf,Libre Inversión,5000000.0,12.0,23.4,24.79,1.77,592914.0,31866.0,5624780.0,4,469730.0,67996.0,3654.0,398080.0,3448689.0
4,BBVA_01_Libre_Inversion_05M_01A_20020317.pdf,Libre Inversión,5000000.0,12.0,23.4,24.79,1.77,592914.0,31866.0,5624780.0,5,469352.0,60959.0,3276.0,405116.0,3043573.0
5,BBVA_01_Libre_Inversion_05M_01A_20020317.pdf,Libre Inversión,5000000.0,12.0,23.4,24.79,1.77,592914.0,31866.0,5624780.0,6,468967.0,53798.0,2891.0,412277.0,2631296.0
6,BBVA_01_Libre_Inversion_05M_01A_20020317.pdf,Libre Inversión,5000000.0,12.0,23.4,24.79,1.77,592914.0,31866.0,5624780.0,7,468575.0,46511.0,2500.0,419564.0,2211732.0
7,BBVA_01_Libre_Inversion_05M_01A_20020317.pdf,Libre Inversión,5000000.0,12.0,23.4,24.79,1.77,592914.0,31866.0,5624780.0,8,468177.0,39095.0,2101.0,426981.0,1784751.0
8,BBVA_01_Libre_Inversion_05M_01A_20020317.pdf,Libre Inversión,5000000.0,12.0,23.4,24.79,1.77,592914.0,31866.0,5624780.0,9,467771.0,31547.0,1696.0,434528.0,1350223.0
9,BBVA_01_Libre_Inversion_05M_01A_20020317.pdf,Libre Inversión,5000000.0,12.0,23.4,24.79,1.77,592914.0,31866.0,5624780.0,10,467358.0,23867.0,1283.0,442209.0,908015.0


# 05. Union dataframes

In [81]:
df_credito_hipotecario.head(3)

,Archivo,Tipo de crédito,Plazo (años),Tasa de interés (%),VTU (%),Valor solicitado,Total intereses,Total seguro de vida,Total seguro incendio/terremoto,Total a pagar + seguros,Mes,Capital,Intereses,Cuota sin seguros,Seguro de vida,Seguro incendio/terremoto,Cuota con seguros,Saldo
0,BBVA_02_Hipotecario_Vivienda_100M_05A_20020321...,Crédito hipotecario,5.0,1286.0,1441.0,100000000.0,33949303.0,587971.0,3570043.0,133949303.0,1,1219235.0,1013248.0,2232483.0,17325.0,59501.0,2309309.0,98780766.0
1,BBVA_02_Hipotecario_Vivienda_100M_05A_20020321...,Crédito hipotecario,5.0,1286.0,1441.0,100000000.0,33949303.0,587971.0,3570043.0,133949303.0,2,1231588.0,1000894.0,2232483.0,17114.0,59501.0,2309097.0,97549177.0
2,BBVA_02_Hipotecario_Vivienda_100M_05A_20020321...,Crédito hipotecario,5.0,1286.0,1441.0,100000000.0,33949303.0,587971.0,3570043.0,133949303.0,3,1244067.0,988415.0,2232483.0,16900.0,59501.0,2308884.0,96305110.0


In [80]:
df_libre_inversion.head(3)

,Archivo,Tipo de crédito,Valor del crédito,Plazo del crédito,Tasa Efectiva Anual (E.A) desde,VTU Porcentual,Tasa Mensual Vencida (M.V) desde,Valor por intereses,Valor por seguro de vida,Total,Mes,Cuota Mensual,Intereses mes vencido,Seguro de vida,Abono a capital,Saldo
0,BBVA_01_Libre_Inversion_05M_01A_20020317.pdf,Libre Inversión,5000000.0,12.0,23.4,24.79,1.77,592914.0,31866.0,5624780.0,1,470825.0,88380.0,4750.0,377695.0,4622305.0
1,BBVA_01_Libre_Inversion_05M_01A_20020317.pdf,Libre Inversión,5000000.0,12.0,23.4,24.79,1.77,592914.0,31866.0,5624780.0,2,470467.0,81704.0,4391.0,384371.0,4237934.0
2,BBVA_01_Libre_Inversion_05M_01A_20020317.pdf,Libre Inversión,5000000.0,12.0,23.4,24.79,1.77,592914.0,31866.0,5624780.0,3,470101.0,74910.0,4026.0,391165.0,3846769.0
